# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

* *Дополнил get_closest_hybrid_match и в связи с изменениями вывода подредактировал алгоритм проверки. Вроде сделал всё так, но результаты почти не отличаются от исходных (процент исправленных ошибок немного подрос)*

In [111]:
from collections import Counter
import textdistance
import re
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [2]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [3]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [4]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [ ]:
vocab

In [ ]:
id2word

In [5]:
# Получаем топ 20 похожих по косиносному расстоянию слов
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [6]:
# Получаем топ 20 самых похожих на заданное слово ('text') слов (в базе слов lookup, куда подставляем vocab) по расстоянию Левенштейна
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

# Преобразование частоты встречаемости слова в корпусе в вероятность для исправления ошибки (из алгоритма Норвига)
N = sum(vocab.values())
def P(word, N=N):
    return vocab[word] / N

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    top_words = []
    for i in range(len(closest)):
        min_dist = closest[0][1]
        #print(min_dist)
        if closest[i][1] == min_dist:
            top_words.append(closest[i][0])
            #print(top_words)

    
    return max(top_words, key=P)


def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [7]:
def get_closest_hybrid_match_0(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

In [8]:
get_closest_hybrid_match_0('обака', X, vec)

[('собака', 0.8333333333333334), ('робака', 0.8333333333333334), ('бака', 0.8)]

In [9]:
get_closest_hybrid_match('обака', X, vec)

'собака'

In [112]:
from tqdm.notebook import tqdm
from string import punctuation
punctuation += "«»—…“”"
punct = set(punctuation)
from string import punctuation

In [11]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [12]:
# Проверяем качество
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec))
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [13]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8556278139069535
0.48835403726708076
0.09004249454461927


## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [113]:
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [114]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()

In [23]:
bad[:1]

['Симпатичнейшое шпионское устройство, такой себе гламурный фотоаппарат девушки Бонда - миниатюрная модель камеры Superheadz Clap Camera.']

In [121]:
# Составляем словарь правильных слов
def true_dict_creation(true):
    words = []
    for sentense in true:
        a = sentense.lower().split()
        for word in a:
            words.append(word)
    true_dict = Counter(words)
    return true_dict

def del_dict_creation(true_dict):
    del_dict = dict()
    for word in true_dict:
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        for delete in deletes:
            del_dict[delete] = word
    return del_dict

def max_matching(text, true_dict):
    N = sum(true_dict.values())
    return true_dict[text] / N

def changing_words(word, true):
    true_dict = true_dict_creation(true)
    del_dict = del_dict_creation(true_dict)
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    matching_words_dict = dict()
    for delete in deletes:
        if delete in del_dict:
            matching_words_dict[del_dict[delete]] = max_matching(del_dict[delete], true_dict)
    sorted_matching_dict = dict(sorted(matching_words_dict.items(), key=lambda item: item[1], reverse=True))
    if sorted_matching_dict:
        best_matching_word, best_matching_word_coef = next(iter(sorted_matching_dict.items()))
        return best_matching_word
    else:
        return word

            

In [122]:
changing_words('нто', true)

'нет'

In [117]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [123]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], changing_words(pair[1], true))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [124]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6710355177588795
0.28726708074534163
0.2721947858045251
